# W11 — Statystyka opisowa w Pythonie

**Programowanie w Pythonie II** | Politechnika Opolska  
**Temat:** Miary centralne, rozproszenie, korelacja, scipy.stats, outliery  
**Dataset:** HR — 200 pracowników (wynagrodzenia, staż, dział)

---

## Mapa notebooka
1. Setup i generowanie danych HR
2. Miary tendencji centralnej (średnia, mediana, dominanta)
3. Miary rozproszenia (std, IQR, percentyle)
4. Korelacja Pearsona i Spearmana
5. scipy.stats — describe, skośność, kurtoza
6. Wykrywanie outlierów — IQR i z-score
7. Wpływ outlierów na statystyki

## 1. Setup i generowanie danych HR

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

print(f"numpy:  {np.__version__}")
print(f"pandas: {pd.__version__}")
import scipy
print(f"scipy:  {scipy.__version__}")

In [ ]:
# Generowanie realistycznego datasetu HR
np.random.seed(42)
n = 200

dzialy = np.random.choice(
    ['IT', 'Sprzedaż', 'HR', 'Marketing', 'Finanse'], n,
    p=[0.30, 0.25, 0.15, 0.20, 0.10]
)

# Staż z rozkładu gamma (prawostronnie skośny — realistyczny)
staz = np.random.gamma(shape=3, scale=2, size=n).clip(0.5, 20).round(1)

# Wynagrodzenie: baza wg działu + premia za staż + szum
baza = {'IT': 9000, 'Sprzedaż': 7000, 'HR': 6500, 'Marketing': 7500, 'Finanse': 8500}
wynagrodzenie = np.array([
    baza[d] + staz[i] * 300 + np.random.normal(0, 1200)
    for i, d in enumerate(dzialy)
]).clip(4000, 25000).round(-2)

# 5 celowych outlierów (błędy danych / specjalne kontrakty)
wynagrodzenie[np.random.choice(n, 5, replace=False)] = np.random.choice(
    [2000, 2500, 35000, 40000, 38000], 5, replace=False
)

df = pd.DataFrame({
    'dział': dzialy,
    'staż_lat': staz,
    'wynagrodzenie': wynagrodzenie,
    'wiek': (25 + staz + np.random.normal(0, 3, n)).clip(22, 65).round().astype(int),
    'ocena_roczna': np.random.choice([1, 2, 3, 4, 5], n, p=[0.05, 0.10, 0.40, 0.35, 0.10])
})

print(f"Dataset HR: {df.shape[0]} pracowników, {df.shape[1]} kolumn")
print(f"Działy: {df['dział'].value_counts().to_dict()}")
print()
df.head()

## 2. Miary tendencji centralnej

**Pytanie biznesowe:** Jakie jest "typowe" wynagrodzenie w firmie?

In [ ]:
placa = df['wynagrodzenie']

srednia   = placa.mean()
mediana   = placa.median()
dominanta = placa.mode()[0]

print("=" * 45)
print("   MIARY TENDENCJI CENTRALNEJ")
print("=" * 45)
print(f"  Średnia:    {srednia:>10,.0f} PLN")
print(f"  Mediana:    {mediana:>10,.0f} PLN")
print(f"  Dominanta:  {dominanta:>10,.0f} PLN")
print("=" * 45)
print()
if mediana < srednia:
    roznica = srednia - mediana
    print(f"Mediana < Średnia (różnica: {roznica:,.0f} PLN)")
    print("→ Rozkład PRAWOSTRONNIE SKOŚNY")
    print("→ Outlierzy wysokich pensji podnoszą średnią")
    print("→ Mediana jest bezpieczniejszą miarą 'typowego' wynagrodzenia")

In [ ]:
# Wizualizacja: histogram z trzema miarami centralnymi
fig, ax = plt.subplots(figsize=(11, 5))

ax.hist(placa, bins=30, color='steelblue', edgecolor='white', alpha=0.85)
ax.axvline(srednia, color='red', lw=2.5, linestyle='--',
           label=f'Średnia: {srednia:,.0f} PLN')
ax.axvline(mediana, color='green', lw=2.5, linestyle='-',
           label=f'Mediana: {mediana:,.0f} PLN')
ax.axvline(dominanta, color='orange', lw=2.5, linestyle=':',
           label=f'Dominanta: {dominanta:,.0f} PLN')

ax.set_title('Rozkład wynagrodzeń — średnia, mediana, dominanta', fontsize=13)
ax.set_xlabel('Wynagrodzenie (PLN)', fontsize=11)
ax.set_ylabel('Liczba pracowników', fontsize=11)
ax.legend(fontsize=11)
ax.annotate('Outlierzy → podnoszą średnią',
            xy=(35000, 2), xytext=(22000, 10),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=10, color='gray')

plt.tight_layout()
plt.show()
plt.close()

## 3. Miary rozproszenia i percentyle

**Pytanie biznesowe:** Jak bardzo zróżnicowane są wynagrodzenia? Gdzie jest 50% środkowych pracowników?

In [ ]:
odch_std = placa.std()
wariancja = placa.var()
q1 = placa.quantile(0.25)
q3 = placa.quantile(0.75)
iqr = q3 - q1
rozstep = placa.max() - placa.min()

print("=" * 45)
print("      MIARY ROZPROSZENIA")
print("=" * 45)
print(f"  Odchylenie std:   {odch_std:>10,.0f} PLN")
print(f"  Wariancja:        {wariancja:>10,.0f} PLN²")
print(f"  Q1 (P25):         {q1:>10,.0f} PLN")
print(f"  Q3 (P75):         {q3:>10,.0f} PLN")
print(f"  IQR (Q3 − Q1):    {iqr:>10,.0f} PLN")
print(f"  Rozstęp (max−min):{rozstep:>10,.0f} PLN")
print("=" * 45)
print()
print("Interpretacja:")
print(f"  50% pracowników zarabia między {q1:,.0f} a {q3:,.0f} PLN (IQR={iqr:,.0f})")
print(f"  Duży rozstęp ({rozstep:,.0f}) wynika z outlierów (min={placa.min():,.0f}, max={placa.max():,.0f})")

In [ ]:
# Statystyki per dział
dzialy_stats = df.groupby('dział', observed=True)['wynagrodzenie'].agg([
    'mean', 'median', 'std', 'min', 'max',
    lambda x: x.quantile(0.75) - x.quantile(0.25)
]).round(0)
dzialy_stats.columns = ['Średnia', 'Mediana', 'Std', 'Min', 'Max', 'IQR']
dzialy_stats = dzialy_stats.sort_values('Mediana', ascending=False)

print("Wynagrodzenia per dział (posortowane wg mediany):")
print(dzialy_stats.to_string())

In [ ]:
# Percentyle
percentyle_lista = [10, 25, 50, 75, 90, 95, 99]

print("=" * 38)
print("      PERCENTYLE WYNAGRODZEŃ")
print("=" * 38)
for p in percentyle_lista:
    val = np.percentile(placa, p)
    bar = '█' * int(p / 5)
    print(f"  P{p:3d}: {val:>10,.0f} PLN  {bar}")
print("=" * 38)

# Procent pracowników powyżej 12 000
powyzej_12k = (placa > 12000).mean() * 100
print(f"\nPracownicy z wynagrodzeniem > 12 000 PLN: {powyzej_12k:.1f}%")

In [ ]:
# Wizualizacja: histogram z percentylami + boxplot per dział
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lewy: histogram z liniami percentyli
axes[0].hist(placa, bins=30, color='steelblue', edgecolor='white', alpha=0.85)
kolory_p = [(25, 'green', 'Q1'), (50, 'orange', 'Mediana'), (75, 'red', 'Q3')]
for p_val, kolor, etykieta in kolory_p:
    val = np.percentile(placa, p_val)
    axes[0].axvline(val, color=kolor, lw=2, linestyle='--',
                    label=f'{etykieta}: {val:,.0f}')
axes[0].set_title('Rozkład wynagrodzeń z kwartylami')
axes[0].set_xlabel('Wynagrodzenie (PLN)')
axes[0].set_ylabel('Liczba pracowników')
axes[0].legend(fontsize=9)

# Prawy: boxplot per dział
dzialy_kolejnosc = dzialy_stats.index.tolist()
dane_boxplot = [df[df['dział'] == d]['wynagrodzenie'].values
                for d in dzialy_kolejnosc]
bp = axes[1].boxplot(dane_boxplot, labels=dzialy_kolejnosc, patch_artist=True,
                     medianprops=dict(color='red', linewidth=2))
kolory_box = ['#4e79a7', '#f28e2b', '#59a14f', '#e15759', '#76b7b2']
for patch, kolor in zip(bp['boxes'], kolory_box):
    patch.set_facecolor(kolor)
    patch.set_alpha(0.7)
axes[1].axhline(placa.median(), color='navy', linestyle=':', lw=1.5,
                label=f'Mediana globalna: {placa.median():,.0f}')
axes[1].set_title('Wynagrodzenie per dział')
axes[1].set_xlabel('Dział')
axes[1].set_ylabel('Wynagrodzenie (PLN)')
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()
plt.close()

## 4. Korelacja Pearsona i Spearmana

**Pytanie biznesowe:** Co wpływa na wysokość wynagrodzenia — staż, wiek czy ocena roczna?

> **Ważne:** korelacja ≠ przyczynowość!

In [ ]:
# Korelacja Pearsona — wymagania: liniowość, rozkład normalny
r_staz, p_staz = stats.pearsonr(df['staż_lat'], df['wynagrodzenie'])
r_wiek, p_wiek = stats.pearsonr(df['wiek'], df['wynagrodzenie'])
r_ocena, p_ocena = stats.pearsonr(df['ocena_roczna'], df['wynagrodzenie'])

print("=" * 62)
print("   KORELACJA PEARSONA z wynagrodzeniem")
print("=" * 62)
print(f"{'Zmienna':<20} {'r':>8} {'p-value':>12} {'Istotność':>12}")
print("-" * 62)
for nazwa, r, p in [
    ('staż_lat', r_staz, p_staz),
    ('wiek', r_wiek, p_wiek),
    ('ocena_roczna', r_ocena, p_ocena)
]:
    istotnosc = '*** p<0.001' if p < 0.001 else ('* p<0.05' if p < 0.05 else 'n.s.')
    print(f"{nazwa:<20} {r:>8.4f} {p:>12.4f} {istotnosc:>12}")
print("=" * 62)

In [ ]:
# Korelacja Spearmana — odporna na outliery i nieliniowość
rho_staz, p_rho_staz = stats.spearmanr(df['staż_lat'], df['wynagrodzenie'])

print("Porównanie Pearson vs Spearman dla staż–wynagrodzenie:")
print(f"  Pearson  r   = {r_staz:.4f}  (zakłada liniowość, wrażliwy na outliery)")
print(f"  Spearman rho = {rho_staz:.4f}  (rangi, odporny na outliery)")
print(f"  Różnica: {abs(rho_staz - r_staz):.4f}")
print()
if rho_staz > r_staz:
    print("Spearman wyższy niż Pearson → outlierzy zaburzają liniowość.")
    print("Spearman poprawnie wychwytuje monotoniczną zależność (im więcej stażu → wyższe płace).")

In [ ]:
# Macierz korelacji Pearsona
corr = df[['staż_lat', 'wynagrodzenie', 'wiek', 'ocena_roczna']].corr()
print("Macierz korelacji Pearsona:")
print(corr.round(3))

In [ ]:
# Wizualizacja: scatter z trendem (staż vs wynagrodzenie)
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: staż vs wynagrodzenie
axes[0].scatter(df['staż_lat'], df['wynagrodzenie'],
                alpha=0.5, color='steelblue', s=45, edgecolors='none')
z = np.polyfit(df['staż_lat'], df['wynagrodzenie'], 1)
p_poly = np.poly1d(z)
x_line = np.linspace(df['staż_lat'].min(), df['staż_lat'].max(), 100)
axes[0].plot(x_line, p_poly(x_line), 'r--', lw=2,
             label=f'Trend (r={r_staz:.2f}, rho={rho_staz:.2f})')
axes[0].set_title('Staż pracy vs Wynagrodzenie')
axes[0].set_xlabel('Staż pracy (lata)')
axes[0].set_ylabel('Wynagrodzenie (PLN)')
axes[0].legend(fontsize=10)

# Prawy: ocena roczna vs wynagrodzenie
axes[1].scatter(df['ocena_roczna'] + np.random.uniform(-0.2, 0.2, len(df)),
                df['wynagrodzenie'],
                alpha=0.4, color='coral', s=45, edgecolors='none')
z2 = np.polyfit(df['ocena_roczna'], df['wynagrodzenie'], 1)
p_poly2 = np.poly1d(z2)
x_line2 = np.linspace(1, 5, 100)
axes[1].plot(x_line2, p_poly2(x_line2), 'b--', lw=2,
             label=f'Trend (r={r_ocena:.2f}) — brak związku')
axes[1].set_title('Ocena roczna vs Wynagrodzenie')
axes[1].set_xlabel('Ocena roczna (1–5)')
axes[1].set_ylabel('Wynagrodzenie (PLN)')
axes[1].set_xticks([1, 2, 3, 4, 5])
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
# Przykład z marketingiem: silna korelacja
np.random.seed(100)
n_mkt = 50
marketing_spend = np.random.uniform(10000, 100000, n_mkt)
revenue = marketing_spend * 5.2 + np.random.normal(0, 30000, n_mkt)
revenue = revenue.clip(0)

r_mkt, p_mkt = stats.pearsonr(marketing_spend, revenue)

fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(marketing_spend / 1000, revenue / 1000,
           alpha=0.65, color='darkorange', s=55, edgecolors='white', linewidths=0.5)
z_mkt = np.polyfit(marketing_spend, revenue, 1)
p_mkt_poly = np.poly1d(z_mkt)
x_mkt = np.linspace(marketing_spend.min(), marketing_spend.max(), 100)
ax.plot(x_mkt / 1000, p_mkt_poly(x_mkt) / 1000, 'navy', lw=2,
        label=f'Trend liniowy (r={r_mkt:.2f})')
ax.set_title('Wydatki na marketing vs Przychody (n=50 miesięcy)', fontsize=12)
ax.set_xlabel('Wydatki marketingowe (tys. PLN)')
ax.set_ylabel('Przychody (tys. PLN)')
ax.legend(fontsize=11)
ax.text(0.05, 0.88, f'r = {r_mkt:.4f}\np = {p_mkt:.2e}\nSilna korelacja pozytywna',
        transform=ax.transAxes, fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()
plt.close()

print(f"\nMarekting spend vs Revenue:")
print(f"  Pearson r = {r_mkt:.4f}")
print(f"  p-value   = {p_mkt:.2e}")
print("\nUWAGA: silna korelacja ≠ przyczynowość!")
print("Aby potwierdzić wpływ reklamy na sprzedaż → potrzebny test A/B (W12).")

## 5. scipy.stats — describe, skośność, kurtoza

Jeden `stats.describe()` daje 6 miar naraz — szybki przegląd każdego datasetu.

In [ ]:
# scipy.stats.describe — pełny opis rozkładu
opis = stats.describe(df['wynagrodzenie'])

print("=" * 48)
print("   scipy.stats.describe — wynagrodzenie")
print("=" * 48)
print(f"  Liczba obserwacji: {opis.nobs}")
print(f"  Min:               {opis.minmax[0]:>12,.0f} PLN")
print(f"  Max:               {opis.minmax[1]:>12,.0f} PLN")
print(f"  Średnia:           {opis.mean:>12,.0f} PLN")
print(f"  Wariancja:         {opis.variance:>12,.0f} PLN²")
print(f"  Skośność:          {opis.skewness:>12.4f}")
print(f"  Kurtoza:           {opis.kurtosis:>12.4f}")
print("=" * 48)
print()

# Interpretacja skośności
sk = opis.skewness
if abs(sk) < 0.5:
    interp_sk = "prawie symetryczny"
elif abs(sk) < 1.0:
    interp_sk = "umiarkowanie skośny"
else:
    interp_sk = "silnie skośny"
kierunek = "PRAWOSTRONNIE" if sk > 0 else "LEWOSTRONNIE"
print(f"Skośność = {sk:.2f} → {kierunek} {interp_sk}")
print(f"  (outlierzy wysokich pensji ciągną rozkład w prawo)")
print()

# Interpretacja kurtozy (nadmiarowej, Fisher)
ku = opis.kurtosis
print(f"Kurtoza = {ku:.2f} → rozkład LEPTOKURTYCZNY (spiczasty, grube ogony)")
print(f"  (rozkład normalny ma kurtozę = 0; tu: {ku:.1f} — skrajne wartości dużo częstsze)")

In [ ]:
# scipy.stats.describe per dział — porównanie skośności
print("Skośność i kurtoza per dział:")
print(f"{'Dział':<12} {'n':>5} {'Skośność':>12} {'Kurtoza':>12} {'Interpretacja'}")
print("-" * 65)

for dzial in ['IT', 'Finanse', 'Marketing', 'HR', 'Sprzedaż']:
    podzb = df[df['dział'] == dzial]['wynagrodzenie']
    opis_d = stats.describe(podzb)
    sk = opis_d.skewness
    interp = "SILNIE prawosk." if sk > 1 else ("Prawosk." if sk > 0.5 else
              ("Lewosk." if sk < -0.5 else "Symetryczny"))
    print(f"{dzial:<12} {opis_d.nobs:>5} {sk:>12.3f} {opis_d.kurtosis:>12.3f}  {interp}")

In [ ]:
# Wizualizacja skośności i percentyli
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: porównanie skośności: wynagrodzenie vs staż
for i, (kol, kolor, tytul) in enumerate([
    ('wynagrodzenie', 'steelblue', f'Wynagrodzenie (skośność={stats.skew(df["wynagrodzenie"]):.2f})'),
    ('staż_lat', 'coral', f'Staż lat (skośność={stats.skew(df["staż_lat"]):.2f})')
]):
    dane = df[kol]
    dane_norm = (dane - dane.min()) / (dane.max() - dane.min())
    axes[0].hist(dane_norm, bins=20, alpha=0.6, label=tytul,
                 color=kolor, edgecolor='white')
axes[0].set_title('Skośność: wynagrodzenie vs staż (znormalizowane)')
axes[0].set_xlabel('Wartość znormalizowana [0-1]')
axes[0].set_ylabel('Liczba')
axes[0].legend(fontsize=9)

# Prawy: diagram pudełkowy z percentylami wynagrodzenia
pct = [10, 25, 50, 75, 90]
wartosci = [np.percentile(placa, p) for p in pct]
kolory_pct = ['#d62728', '#ff7f0e', '#2ca02c', '#ff7f0e', '#d62728']
bars = axes[1].barh([f'P{p}' for p in pct], wartosci,
                     color=kolory_pct, alpha=0.75, edgecolor='white')
for bar, val in zip(bars, wartosci):
    axes[1].text(val + 200, bar.get_y() + bar.get_height() / 2,
                 f'{val:,.0f} PLN', va='center', fontsize=10)
axes[1].set_title('Percentyle wynagrodzeń')
axes[1].set_xlabel('Wynagrodzenie (PLN)')
axes[1].set_xlim(0, placa.max() * 1.15)

plt.tight_layout()
plt.show()
plt.close()

## 6. Wykrywanie wartości odstających — IQR i z-score

**Pytanie biznesowe:** Którzy pracownicy mają anomalne wynagrodzenia? Czy to błąd danych, czy specjalne kontrakty?

In [ ]:
# Metoda 1: IQR (Tukey's fences)
q1_out = placa.quantile(0.25)
q3_out = placa.quantile(0.75)
iqr_out = q3_out - q1_out
dolna = q1_out - 1.5 * iqr_out
gorna = q3_out + 1.5 * iqr_out

maska_iqr = (placa < dolna) | (placa > gorna)
outliery_iqr = df[maska_iqr]

print("=" * 52)
print("   WYKRYWANIE OUTLIERÓW — METODA IQR")
print("=" * 52)
print(f"  Q1 = {q1_out:,.0f} PLN")
print(f"  Q3 = {q3_out:,.0f} PLN")
print(f"  IQR = {iqr_out:,.0f} PLN")
print(f"  Dolna granica: Q1 − 1.5×IQR = {dolna:,.0f} PLN")
print(f"  Górna granica: Q3 + 1.5×IQR = {gorna:,.0f} PLN")
print(f"  Outlierów: {maska_iqr.sum()} z {len(df)} ({maska_iqr.mean()*100:.1f}%)")
print("=" * 52)
print()
print("Szczegóły outlierów:")
print(outliery_iqr[['dział', 'staż_lat', 'wynagrodzenie']].to_string())

In [ ]:
# Metoda 2: z-score
z_scores = np.abs(stats.zscore(df['wynagrodzenie']))
prog_z = 3.0
maska_z = z_scores > prog_z
outliery_z = df[maska_z]

print("=" * 52)
print("  WYKRYWANIE OUTLIERÓW — METODA Z-SCORE")
print("=" * 52)
print(f"  Próg: |z-score| > {prog_z}")
print(f"  Outlierów: {maska_z.sum()} z {len(df)} ({maska_z.mean()*100:.1f}%)")
print("=" * 52)
print()
print("Szczegóły outlierów (z-score > 3):")
print(outliery_z[['dział', 'staż_lat', 'wynagrodzenie']].to_string())
print()
print("Porównanie metod:")
print(f"  IQR:     {maska_iqr.sum()} outlierów (bardziej czuła, nie zakłada normalności)")
print(f"  z-score: {maska_z.sum()} outlierów (mniej czuła, zakłada rozkład normalny)")

In [ ]:
# Wizualizacja outlierów
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lewy: scatter z zaznaczonymi outlierami IQR
normalne = df[~maska_iqr]
axes[0].scatter(normalne.index, normalne['wynagrodzenie'],
                alpha=0.5, color='steelblue', s=30, label='Normalne', zorder=2)
axes[0].scatter(outliery_iqr.index, outliery_iqr['wynagrodzenie'],
                alpha=0.9, color='red', s=100, marker='X', label='Outlierzy IQR', zorder=3)
axes[0].axhline(dolna, color='orange', linestyle='--', lw=1.5, label=f'Dolna granica: {dolna:,.0f}')
axes[0].axhline(gorna, color='orange', linestyle='--', lw=1.5, label=f'Górna granica: {gorna:,.0f}')
axes[0].set_title(f'Outliery metodą IQR (n={maska_iqr.sum()})')
axes[0].set_xlabel('Indeks pracownika')
axes[0].set_ylabel('Wynagrodzenie (PLN)')
axes[0].legend(fontsize=9)

# Prawy: histogram z zaznaczonymi outlierami
axes[1].hist(df[~maska_iqr]['wynagrodzenie'], bins=25,
             color='steelblue', edgecolor='white', alpha=0.8, label='Normalne')
axes[1].hist(outliery_iqr['wynagrodzenie'], bins=5,
             color='red', edgecolor='white', alpha=0.9, label='Outlierzy IQR')
axes[1].axvline(dolna, color='orange', lw=2, linestyle='--')
axes[1].axvline(gorna, color='orange', lw=2, linestyle='--')
axes[1].set_title('Rozkład z outlierami (czerwone)')
axes[1].set_xlabel('Wynagrodzenie (PLN)')
axes[1].set_ylabel('Liczba')
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()
plt.close()

## 7. Wpływ outlierów na statystyki

**Pytanie kluczowe:** Które miary są odporne na wartości odstające?

In [ ]:
# Porównanie statystyk z outlierami i bez
z_out   = df['wynagrodzenie']
bez_out = df[~maska_iqr]['wynagrodzenie']

print("=" * 62)
print("   WPŁYW OUTLIERÓW NA STATYSTYKI")
print("=" * 62)
print(f"{'Miara':<20} {'Z outlierami':>14} {'Bez outlierów':>14} {'Zmiana':>10}")
print("-" * 62)

miary = [
    ('Średnia', z_out.mean(), bez_out.mean()),
    ('Mediana', z_out.median(), bez_out.median()),
    ('Std',     z_out.std(), bez_out.std()),
    ('IQR',
     z_out.quantile(0.75) - z_out.quantile(0.25),
     bez_out.quantile(0.75) - bez_out.quantile(0.25)),
]

for nazwa, z_val, bez_val in miary:
    zmiana = z_val - bez_val
    print(f"{nazwa:<20} {z_val:>14,.0f} {bez_val:>14,.0f} {zmiana:>+10,.0f}")

print("=" * 62)
print()
print("Wnioski:")
print("  Mediana: stabilna — różnica tylko ~50 PLN")
print("  Średnia: niestabilna — różnica ~400 PLN")
print("  Std: bardzo wrażliwe — różnica >2000 PLN (ponad 2×!)")
print("  IQR: stabilny — mierzy środkowe 50%, ignoryje krańce")

In [ ]:
# Podsumowanie wizualne: czułość miar na outliery
miary_nazwy = ['Średnia', 'Mediana', 'Std', 'IQR']
wartosci_z   = [z_out.mean(), z_out.median(), z_out.std(),
                z_out.quantile(0.75) - z_out.quantile(0.25)]
wartosci_bez = [bez_out.mean(), bez_out.median(), bez_out.std(),
                bez_out.quantile(0.75) - bez_out.quantile(0.25)]

x = np.arange(len(miary_nazwy))
szerokosc = 0.35

fig, ax = plt.subplots(figsize=(9, 5))
bars1 = ax.bar(x - szerokosc/2, wartosci_z, szerokosc,
               label='Z outlierami', color='salmon', edgecolor='white')
bars2 = ax.bar(x + szerokosc/2, wartosci_bez, szerokosc,
               label='Bez outlierów', color='steelblue', edgecolor='white')

for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
            f'{bar.get_height():,.0f}', ha='center', va='bottom', fontsize=8)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
            f'{bar.get_height():,.0f}', ha='center', va='bottom', fontsize=8)

ax.set_title('Czułość miar statystycznych na outliery', fontsize=12)
ax.set_xlabel('Miara statystyczna')
ax.set_ylabel('Wartość (PLN)')
ax.set_xticks(x)
ax.set_xticklabels(miary_nazwy)
ax.legend()

plt.tight_layout()
plt.show()
plt.close()

print("\nKLUCZOWY WNIOSEK:")
print("  Dane z outlierami → MEDIANA i IQR są bezpieczne")
print("  ŚREDNIA i STD — bardzo wrażliwe, mogą wprowadzać w błąd")

## Podsumowanie — kluczowe zasady

| Sytuacja | Zalecana miara |
|----------|----------------|
| Dane skośne lub z outlierami | **Mediana** (nie średnia) |
| Rozproszenie przy outlierach | **IQR** (nie std) |
| Zależność liniowa, normalne dane | **Pearson r** |
| Dane niespełniające normalności / outliery | **Spearman rho** |
| Szybki przegląd rozkładu | **scipy.stats.describe()** |
| Wykrywanie anomalii | **IQR** (pierwsza metoda) + z-score |

> **Zawsze pamiętaj:** korelacja ≠ przyczynowość!  
> **Outlierów nie kasujemy automatycznie** — najpierw weryfikacja: błąd danych czy realny przypadek?

**Następny wykład (W12):** Testy hipotez — t-test, Mann-Whitney, chi-kwadrat, symulacja testu A/B.